In [2]:
!pip install mtcnn

     |████████████████████████████████| 2.3 MB 980 kB/s eta 0:00:01


In [3]:
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from mtcnn.mtcnn import MTCNN
from PIL import Image
import os 


In [22]:
## change the folder name accordingly for training and testing
path = '/Users/swatiraman/Downloads/Hackathon_DL/train/'

folders = os.listdir(path)
#folders = folders[1:] ## [1:] to remove .ds_store folder if it is made automatically otherwise just use folder
detector = MTCNN()
## Iterate over the folder and detect and crop faces and save them in respective folder
for subs in folders:
    for files in os.listdir(path+subs):
        try:
            if 'fake' in path+subs and 'jpg' in path+subs+files:
                print(path+subs+'/'+files)
                pixels = pyplot.imread(path+subs+'/'+files)
                faces = detector.detect_faces(pixels)
                coordinates = tuple(faces[0]['box'])
                Image.fromarray(pixels).crop(coordinates).save('train/training_fake/'+files)
        except (IndexError or SystemError):
            print('Face Not Found')
        try:
            if 'real' in path+subs and 'jpg' in path+subs+files:
                print(path+subs+'/'+files)
                pixels = pyplot.imread(path+subs+'/'+files)
                faces = detector.detect_faces(pixels)
                coordinates = tuple(faces[0]['box'])
                #Image.fromarray(pixels).crop(coordinates).save('train/training_real/'+files)
        except (IndexError or SystemError):
            print('Face Not Found')

/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/63.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/189.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/77.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/638.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/604.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/162.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/176.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/88.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/610.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/348.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/360.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/406.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/412.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/374.jpg
/Users/swatiraman/Downloads/Hackathon

/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/66.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/72.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/167.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/601.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/99.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/615.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/173.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/629.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/403.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/365.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/371.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/417.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/359.jpg
/Users/swatiraman/Downloads/Hackathon_DL/train/training_fake/588.jpg
/Users/swatiraman/Downloads/Hackathon

SystemError: tile cannot extend outside image

In [11]:
from keras.models import Sequential 
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [19]:
model = Sequential()

model.add(Conv2D(32,(3,3),input_shape = (200,200,3),activation = 'relu',padding='same',))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer = 'Adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [21]:
datagen = ImageDataGenerator(rescale=1.0/255.0)

train = datagen.flow_from_directory('/Users/swatiraman/Downloads/Hackathon_DL/train/',
                                    class_mode='binary',
                                    batch_size=64,
                                    target_size=(200,200))

test = datagen.flow_from_directory('/Users/swatiraman/Downloads/Hackathon_DL/test/',
                                    class_mode='binary',
                                    batch_size=64,
                                    target_size=(200,200))
history = model.fit(train,
                              validation_data=(test),
                              epochs = 50,
                              steps_per_epoch=len(train),
                              validation_steps=len(test))

Found 3418 images belonging to 4 classes.
Found 0 images belonging to 0 classes.
Epoch 1/50
51/54 [===========================>..] - ETA: 4s - loss: -1648111.6250 - accuracy: 0.2204

KeyboardInterrupt: 

In [ ]:
model_json = model.to_json()

with open('model.json','w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')

In [ ]:
from keras.models import load_model, model_from_json
from PIL import Image
import json
import numpy as np 

# Load model from Json file 
json_file = open('model.json','r')
loaded_model = json_file.read()
json_file.close()

load_model = model_from_json(loaded_model)
load_model.load_weights('model.h5')

# Load Image 
image = Image.open('real (2772).jpg') ## Test Image Path
im = image.resize((200,200))

im = np.asarray(im)
im = np.reshape(im,(1,im.shape[0],im.shape[1],im.shape[2]))

# Make Prediction 
prediction = load_model.predict(im)
if prediction == 1:
  print('Real Face')
else:
  print('Fake Face')